- 模型应用
  - 模型加载
  - 模型测试集评估
    - 准确率
    - 召回率
    - F1


In [ ]:
# 测试集评论数据测试 1

import time

import torch

import config
import dataset
import model
import run
import word_embedding

model = model.SentimentAnalysisModel().load(config.MODEL_PTH_FILENAME).to(config.DEVICE)
print('test_model[{:s}]'.format(config.MODEL_PTH_FILENAME), )
word_embedding = word_embedding.WordEmbedding().load(config.WORD_EMBEDDING_PTH_FILENAME).to(config.DEVICE)
print('test_word_embedding[{:s}]'.format(config.WORD_EMBEDDING_PTH_FILENAME), )

test_loader = dataset.SentimentAnalysisDataset(data_path=config.TEST_DATA_PATH, word_embedding=word_embedding,
                                               sequence_length=config.SEQUENCE_LENGTH, ).construct_dataloader(
    batch_size=config.TRAIN_BATCH_SIZE)

loss_fn = torch.nn.CrossEntropyLoss()
test_loss, test_macro_precision, test_macro_recall, test_macro_f1 = run.test(model, test_loader, loss_fn, config.DEVICE)

print('test_loss[{:f}]'.format(test_loss),
      'test_macro_precision[{:f}]'.format(test_macro_precision),
      'test_macro_recall[{:f}]'.format(test_macro_recall),
      'test_macro_f1[{:f}]'.format(test_macro_f1),
      time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

In [ ]:
# 测试集评论数据测试 2

import time

import torch
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import config
import dataset
import model
import word_embedding

model = model.SentimentAnalysisModel().load(config.MODEL_PTH_FILENAME).to(config.DEVICE)
print('test_model[{:s}]'.format(config.MODEL_PTH_FILENAME), )
word_embedding = word_embedding.WordEmbedding().load(config.WORD_EMBEDDING_PTH_FILENAME).to(config.DEVICE)
print('test_word_embedding[{:s}]'.format(config.WORD_EMBEDDING_PTH_FILENAME), )

test_dataset = dataset.SentimentAnalysisDataset(data_path=config.TEST_DATA_PATH, word_embedding=word_embedding,
                                                sequence_length=config.SEQUENCE_LENGTH, ).dataset

loss_fn = torch.nn.CrossEntropyLoss()

model.eval()
data, targets = test_dataset.tensors[0].to(config.DEVICE), test_dataset.tensors[1].to(config.DEVICE)
output = model(data)
test_loss = loss_fn(output, targets).item()
predicted = output.argmax(dim=1).cpu().numpy()
labels = targets.cpu().numpy()
print("每个类别的精确率和召回率：\n",
      classification_report(labels, predicted, target_names=dataset.emotion2label.keys()))

test_macro_precision = precision_score(labels, predicted, average='macro', zero_division=0)  # 宏精准率
test_macro_recall = recall_score(labels, predicted, average='macro', zero_division=0)  # 宏召回率
test_macro_f1 = f1_score(labels, predicted, average='macro')  # 宏F1值
print('test_loss[{:f}]'.format(test_loss),
      'test_macro_precision[{:f}]'.format(test_macro_precision),
      'test_macro_recall[{:f}]'.format(test_macro_recall),
      'test_macro_f1[{:f}]'.format(test_macro_f1),
      time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

In [ ]:
# 随机评论数据测试

import config
import model
import dataset
import numpy
import torch
import word_embedding

model = model.SentimentAnalysisModel().load(config.MODEL_PTH_FILENAME).to(config.DEVICE)
word_embedding = word_embedding.WordEmbedding().load(config.WORD_EMBEDDING_PTH_FILENAME).to(config.DEVICE)

contents = ["感觉这个秤非常小巧精致实用。自从看到家人买来这个秤以后，其他的包括秤（包括金器秤）都被闲置了",
            "第三次购买了，还算可以，希望能更耐用一些。",
            "没什想说的，中规中矩，没什么色差",
            "秤盘不平，一边高一边低。这个价位不应该这样了。按键硬，按起来费力。底座盘不平，有一个脚悬空",
            "底座四个点不平，质量不行，本来就是有精度要求，结果买一个底座不平的电子秤。简直是一个笑话",
            "182712798172498172948718924791872489172894817"]

for content in contents:
    vecs = word_embedding.sentences2vecs([content])
    vec = vecs[0]
    vec_padding = dataset.vec_padding(vec, config.SEQUENCE_LENGTH, word_embedding.word2vec_vector_size)
    data = torch.Tensor(numpy.array(vec_padding)).unsqueeze(0).to(config.DEVICE)
    output, _ = model(data)
    predicted = output.argmax(dim=1).cpu().numpy()
    print(content, predicted[0], dataset.label2emotion.get(predicted[0]))